<img src="https://raw.githubusercontent.com/AdrianoPereira/CAP421/main/lectures/homework11/images/cover-homework11.png" style="width: 100%;">

### OVERVIEW
<hr />

This notebook contains exercise 11 (optional exercise 02). This exercise was proposed by Professor Valdivino Santiago Júnior in the course CAP421 - Deep Learning offered in the Postgraduate Program in Applied Computing at the National Institute for Space Research.

**Exercise optional 02:**
> Explain why the "new" proposed DenseNet-83 has more trainable parameters and demands more memory (parameters) than the DenseNet-121. Create a simple network with some dense blocks (enventually transition layers), calculate the number os trainable parameters and related memory to solve this exercise.


**Author:** Adriano P. Almeida <<adriano.almeida@inpe.br>>
<br>
**Created on:** 12 November, 2021
<br /><br />

<a href="#">
    <img style="float: left; margin-right: 10px;" src="https://colab.research.google.com/assets/colab-badge.svg" />
</a>

<a href="https://github.com/AdrianoPereira/CAP421/tree/main/lectures/homework11">
    <img style="float: left; margin-right: 10px;" src="https://img.shields.io/badge/GitHub-Open%20Repository-lightgrey?logo=github" />
</a>

## 1. DenseNet
<hr />

A DenseNet é um arquitetura de rede neural convolucional em que as camadas ocultas possuem conexões com as camadas subsequentes, com um certo grau de persistência [(HUANG, 2017)](#huang_densely_2017). Essa característica é semelhante a que esta presente na arquitetura ResNet [(HE, 2016)](#he_deep_2016), porém, existem diferenças em como essas conexões são feitas. A ResNet realiza uma operação de soma com as informações disponíveis nas camadas, que pode ser descrita como $a^{l} = g(a^{[l+1]}+a^{[1]})$, em que $g$ é uma função da taxa de alcance da rede neural. Já a DenseNet realiza uma concatenação das conxeções, descrita como $a^{l} = g(a^{[0]}, a^{[1]}, ..., a^{[l-1]})$. Essas conexões permitem que a rede não tenha o problema do desaparecimento de gradiente, uma vez que as informações são transferidas para mais de uma camada, como pode ser observado na Figura [1](#densenet). Cada camada possui uma conexão com as camadas subsequentes.

<p>
<img width="60%" src="https://raw.githubusercontent.com/AdrianoPereira/CAP421/main/lectures/homework11/images/densenet.png" />
<br />
<span style="display: block; text-align: center;" id="densenet">
<strong>Figura 1</strong>: Esquema de um bloco da arquitetura DenseNet. <strong>Fonte</strong>:<a href=""#huand_densely_2017">Huan (2017)</a>
</span>
</p>
    
O número de conexões diretas da DenseNet é dado po $\frac{L(L+1)}{2}$, em que $L$ é o número de camadas. Existem versões da DenseNet com diferentes números de camadas, tais como 121, 160, 201, etc. Esse número de camadas é dado por blocos que contém filtros convolucionais, camadas de transição e de classificação. Por exemplo, DenseNet-121 possui 4 blocos com duas camadas convolucionais que totalizam 116 camadas, incluindo suas repetições. As 6 camadas restantes da DenseNet são compostas por suas camadas de transição, que são constituídas por uma camada de convolução e outra de sub-amostragem (*pooling*). A tabela na Figura [2](#table_densenet) mostra as configurações de camadas que compõem as principais versões da DenseNet. Neste exemplo, é utilizado uma imagem do banco de dados ImageNet que possui imagens com resolução de 224$\times$224 e 1000 classes. A primeira coluna mostra o tipo de operação que é aplicado em cada etapa da construção da rede. A segunda coluna mostra a resolução da imagem de saída ao aplicar a operação, essa saída pode ser calculada pela Equação [1](#calc_output), em que $d$ é o número de colunas ou linhas, $p$ é o tamanho da borda (na DenseNet é aplicado uma borda de 3 pixels) e $s$ é a quantidade de pixels utilizada no deslocamento. Sendo assim, utilizando como exemplo a primeira operação no exemplo da tabela da Figura [2](#table_densenet), temos: $d = 224$, $k = 7$ e $s = 2$, que ao aplicar a Equação [1](#calc_output) resulta em 112. As demais colunas da tabela da Figura [2](#table_densenet) mostra as configurações dos blocos para cada versão da DenseNet.

<p>
<img width="90%" src="https://raw.githubusercontent.com/AdrianoPereira/CAP421/main/lectures/homework11/images/table_densenet.png" />
<br />
<span style="display: block; text-align: center;" id="table_densenet">
<strong>Figura 2</strong>: Tabela com as configurações das principais arquiteturas baseadas na DenseNet. <strong>Fonte</strong>: <a href=""#huand_densely_2017">Huan (2017)</a>
</span>
</p>
    
<p id="eq01">
$$
Output = \frac{d+2p-k}{s}+1
$$
<span style="display: block; text-align: center; margin-top: 10px;"><strong>Equação 1</strong>: Cálculo da dimensão da imagem de saída após aplicar a operação nas camadas de convolução.</span>
</p>

### 1.1. Camada de entrada
<hr />

A camada de entrada da DenseNet é formada principalmente por duas camadas, sendo uma de convolução e outra de sub-amostragem (*pooling*), conforme destacado na Figura [3](#input_layer). A camada de convolução possui filtros com o tamanho 7$\times$7 com o deslocamento de 2 pixels. Já a camada de sub-amostragem é definida por uma função de *max pooling* em um *kernel* de tamanho 3$\times$3 e deslocamento de 2 pixels.

<p>
<img width="90%" src="https://raw.githubusercontent.com/AdrianoPereira/CAP421/main/lectures/homework11/images/input_layer.png" />
<br />
<span style="display: block; text-align: center;" id="blocks_densenet">
<strong>Figura 3</strong>: Camada de entrada da DenseNet realçada. <strong>Adaptado de </strong>: <a href=""#huand_densely_2017">Huan (2017)</a>
</span>
</p>
    
A Figura [4](#create_input_layer) mostra o trecho de código em python utilizando a biblioteca [TensorFlow](https://www.tensorflow.org) para formatar a camada de entrada da DenseNet. Primeiramente, é criado um tensor para o shape de entrada dos dados (largura, altura, e canais. E.g. `input_shape = (224, 224, 3)`), em seguida é aplicado o preprocessamento para formatar as bordas da imagem, para enfim, aplicar a camada de convolução. A camada de convolução é definida com 64 filtros com tamanho 7$\times$7 e deslocamento de 2 pixels. Após isso é aplicado uma normalização por lotes e a função de ativação ReLU. Por fim, é aplicada a camada de sub-amostragem por *max pooling*.

<p>
<img width="90%" src="https://raw.githubusercontent.com/AdrianoPereira/CAP421/main/lectures/homework11/images/create_input_layer.png" />
<br />
<span style="display: block; text-align: center;" id="create_input_layer">
<strong>Figura 4</strong>: Função para criar camada de entrada da DenseNet usando o TensorFlow.
</span>
</p>

### 1.2. Blocos densos
<hr />

A DenseNet é formada por 4 blocos densos de convolução que é repetido de acordo com a configuração da arquitetura. A Figura [5](#blocks_densenet) destaca os blocos (linhas) para cada uma das principais arquiteturas (colunas). O bloco é constituído por duas camadas de convolução com um número de filtros que é incrementado a cada construção, de dimensões  1$\times$1 e 3$\times$3. Como pode ser observado, para a DenseNet 121, o primeiro bloco é construído 6 vezes, o segundo bloco 12 vezes, o teceiro bloco é construído 24 vezes e finalmente, o bloco 4 é contruído 16 vezes. Todos esses blocos são conectados em um processo denominado *bottleneck*.

<p>
<img width="90%" src="https://raw.githubusercontent.com/AdrianoPereira/CAP421/main/lectures/homework11/images/blocks.png" />
<br />
<span style="display: block; text-align: center;" id="blocks_densenet">
<strong>Figura 5</strong>: Tabela com os blocos densos realçados para cada arquitetura da DenseNet. <strong>Adaptado de </strong>: <a href=""#huand_densely_2017">Huan (2017)</a>
</span>
</p>
    
Utilizando o TensorFlow, a construção desses blocos e o processo de concatenação pode ser feito através da função `dense_block` no trecho de código da Figura [6](#create_blocks). Utilizando essa função, a criação do bloco denso é criado na chamada da função passando a camada anterior ou a camada de entrada (se for o primeiro bloco) e o número vezes que o bloco será construído. A função `_bottleneck` constrói efetivamente as duas camadas de convolução e faz a concatenação com o bloco anterior. Cada camada é composta por uma normaalização por lotes e a função de ativação ReLU.

<p>
<img width="75%" src="https://raw.githubusercontent.com/AdrianoPereira/CAP421/main/lectures/homework11/images/create_block.png" />
<br />
<span style="display: block; text-align: center;" id="create_blocks">
<strong>Figura 6</strong>: Função para criar blocos da DenseNet usando o TensorFlow.<strong>
</span>
</p>

### 1.3. Camadas de transição
<hr />

Como mencionado, as camadas intermediárias, são formadas por uma camada convolucional com o kernel de tamanho 1$\times$1 e uma camada de sub-amostragem de kernel 2$\times$2 com deslocamento (*stride*) de 2 pixels. A Figura [7](#transition_layers) mostra as camadas de transição realçadas.

<p>
<img width="90%" src="https://raw.githubusercontent.com/AdrianoPereira/CAP421/main/lectures/homework11/images/transition_layers.png" />
<br />
<span style="display: block; text-align: center;" id="transition_layers">
<strong>Figura 7</strong>: Tabela com as camadas de transição realçadas para cada arquitetura da DenseNet. <strong>Adaptado de</strong>: <a href=""#huand_densely_2017">Huan (2017)</a>
</span>
</p>
    
O trecho de código na Figura [8](#create_transition_layer) mostra a função para a criação das camadas de transição utilizando o TensorFlow. Primeiramente é feita a normalização por lotes dos dados e aplicado a função de ativação ReLU, em seguida é criado a camada convolucional de 1$\times$1 e a camada de sub-amostragem com tamanho 2$\times$2 e deslocamento de 2 pixels.

<p>
<img width="75%" src="https://raw.githubusercontent.com/AdrianoPereira/CAP421/main/lectures/homework11/images/create_transition_layer.png" />
<br />
<span style="display: block; text-align: center;" id="create_transition_layer">
<strong>Figura 8</strong>: Função para criar as camadas de transição da DenseNet usando o TensorFlow.<strong>
</span>
</p>

### 1.4. Camada de saída
<hr />

A camada de saída da DenseNet é composta por um camada de sub-amostragem utilizando o *average pooling* com o tamanho do *kernel* de 7$\times$7 e finalmente com uma camada densa totalmente conectada com o número de neurônios equivalente ao total de classes do problema com a função de ativação *softmax*, conforme mostrado na Figura [9](#output_layer).

<p>
<img width="100%" src="https://raw.githubusercontent.com/AdrianoPereira/CAP421/main/lectures/homework11/images/output_layer.png" />
<br />
<span style="display: block; text-align: center;" id="output_layer">
<strong>Figura 9</strong>: Camada de saída da DenseNet destacada.<strong>
</span>
</p>

A Figura [10](#create_output_layer) mostra o trecho de código de uma função para criar a camada de saída da DenseNet utilizando o TensorFlow.
    
<p>
<img width="75%" src="https://raw.githubusercontent.com/AdrianoPereira/CAP421/main/lectures/homework11/images/create_output_layer.png" />
<br />
<span style="display: block; text-align: center;" id="create_output_layer">
<strong>Figura 10</strong>: Função para criar a camada de saída da DenseNet usando o TensorFlow.<strong>
</span>
</p>

## 2. Configurando experimentos
<hr />

Os experimentos executados neste trabalho aplicará duas versões da DenseNet em um problema de classificação. Os modelos serão comparados com relação à quantidade de parâmetros, uso de memória, duração do treinamento e performance.

### 2.1 Importação das bibliotecas e configurações
<hr />

O trecho de código a seguir faz a importação das bibliotecas que serão utilizadas e a definição de variáveis e configurações dos experimentos.

In [66]:
import os
import requests
import tarfile
import numpy as np
import tensorflow as tf
import pickle
import matplotlib.pyplot as plt
import sklearn.metrics as smetrics


PATH = "/home/adriano/CAP421/lectures/homework11"

DATA_PATH = os.path.join(PATH, 'data')
LOG_PATH = os.path.join(PATH, 'logs')
RESULT_PATH = os.path.join(PATH, 'results')
PATH_IMGS = os.path.join(PATH, f"data/images")

if not os.path.exists(DATA_PATH): os.makedirs(DATA_PATH)
if not os.path.exists(RESULT_PATH): os.makedirs(RESULT_PATH)
if not os.path.exists(PATH_IMGS): os.makedirs(PATH_IMGS)

FILENAME = os.path.join(PATH, f"{PATH_IMGS}/imagenette2-320.tgz")
URL = "https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-320.tgz"
CHUNK_SIZE = 1024

# URL = "http://madm.dfki.de/files/sentinel/EuroSAT.zip"

# TRAIN_SIZE = 0.70
# VAL_SIZE = 0.15
# TEST_SIZE = 0.15
# PATH_IMGS = os.path.join(PATH, f"data/images")
# SPATH_IMGS = os.path.join(PATH, f"data/splitted")
# results_path = f"{PATH}/images"
# if not os.path.exists(results_path): os.makedirs(results_path)

In [67]:
def download_imagenette2_320():
    global FILENAME, URL, CHUNK_SIZE
    print(f"Downloading file from {URL}...")
    
    res = requests.get(URL, stream=True)
    
    with open(FILENAME, 'wb') as handle:
        for obj in res.iter_content(chunk_size=CHUNK_SIZE):
            handle.write(obj)
        
        
def extract_imagenette2_320():
    print("Extracting imagenette2-320...")
    global PATH_IMGS, FILENAME
    
    with tarfile.open(FILENAME, 'r:gz') as handle:
        handle.extractall(PATH_IMGS)

download_imagenette2_320()

### 2.1 Carregando dados
<hr />

O Conjunto de dados que será carregado será o [imagenette320 px](https://github.com/fastai/imagenette), que contém um sub-conjunto de dados de 10 classes (*tench, English springer, cassette player, chain saw, church, French horn, garbage truck, gas pump, golf ball, parachute*) do benchmark ImageNet.

### 2.2 Implementando a DenseNet
<hr />

A implementação da DenseNet utilizada neste trabalho será feito utilizando o TensorFlow. Será criado uma função base para gerar os modelos baseados em qualquer arquitetura baseada na DenseNet, no entanto, o código fornece suporte para as três versões já existentes (DenseNet-121, DenseNet-169 e DenseNet-201) e a arquitetura proposta no exercício opcional 1 (DenseNet-83). O código abaixo contém as três funções base para a implementação da DenseNet:

In [36]:
# Importing libraries
import numpy as np
import tensorflow as tf


# Create bottleneck layer
def bottleneck_layer(x, growth_rate, name):
    bn_axis = 3
    x1 = tf.keras.layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                            name=name+'_0_bn')(x)
    x1 = tf.keras.layers.Activation('relu', name=name+'_0_relu')(x1)
    x1 = tf.keras.layers.Conv2D(4 * growth_rate, 1,
                                use_bias=False,
                                name=name+'_0_conv')(x1)
    x1 = tf.keras.layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                            name=name+'_1_bn')(x1)
    x1 = tf.keras.layers.Activation('relu', name=name+'_1_relu')(x1)
    x1 = tf.keras.layers.Conv2D(growth_rate, 3,
                                padding='same',
                                use_bias=False,
                                name=name+'_1_conv')(x1)
    x = tf.keras.layers.Concatenate(axis=bn_axis, name=name+'_concat')([x, x1])
    return x


# Create dense block
def dense_block(x, blocks, name):
    for i in range(blocks):
        x = bottleneck_layer(x, 32, name=name+'_block'+str(i+1))
    return x


# Create transition layer
def transition_layer(x, reduction, name):
    bn_axis = 3
    x = tf.keras.layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                           name=name+'_bn')(x)
    x = tf.keras.layers.Activation('relu', name=name+'_relu')(x)
    x = tf.keras.layers.Conv2D(int(x.shape[bn_axis] * reduction), 1,
                               use_bias=False,
                               name=name+'_conv')(x)
    x = tf.keras.layers.AveragePooling2D(2, strides=2, name=name+'_pool')(x)
    return x

O código abaixo contém a função para a implementação do modelo baseado na DenseNet. E recebe como entrada o nome da arquitetura que será implementada, as dimensões da imagem de entrada, o número de classes e se a camada de saída vai ser implementada.

In [16]:
# Create DenseNet
def densenet(
    architecture='densenet121', input_shape=(224, 224, 3), n_classes=3, 
    include_top=True
):
    VALID_ARCHITECTURES = {
        'densenet83': (3, 6, 18, 12),
        'densenet121': (6, 12, 24, 16), 
        'densenet169': (6, 12, 32, 32), 
        'densenet201': (6, 12, 48, 32),
    }

    if architecture not in VALID_ARCHITECTURES.keys():
        raise ValueError('Invalid architecture name')

    blocks = VALID_ARCHITECTURES[architecture]

    bn_axis = 3
    input_layer = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.ZeroPadding2D(padding=((3, 3), (3, 3)))(input_layer)
    x = tf.keras.layers.Conv2D(64, 7, strides=2, use_bias=False, name='conv1/conv')(x)
    x = tf.keras.layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='conv1/bn')(x)
    x = tf.keras.layers.Activation('relu', name='conv1/relu')(x)
    x = tf.keras.layers.MaxPooling2D(3, strides=2, name='pool1')(x)

    x = dense_block(x, blocks[0], name='conv2')
    x = transition_layer(x, 0.5, name='pool2')
    x = dense_block(x, blocks[1], name='conv3')
    x = transition_layer(x, 0.5, name='pool3')
    x = dense_block(x, blocks[2], name='conv4')
    x = transition_layer(x, 0.5, name='pool4')
    x = dense_block(x, blocks[3], name='conv5')

    x = tf.keras.layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='bn')(x)
    x = tf.keras.layers.Activation('relu', name='relu')(x)

    if include_top:
        x = tf.keras.layers.GlobalAveragePooling2D(name='avg_pool')(x)
        x = tf.keras.layers.Dense(n_classes, activation='softmax', name='prediction')(x)
    else:
        x = tf.keras.layers.GlobalAveragePooling2D(name='avg_pool')(x)

    model = tf.keras.models.Model(input_layer, x, name=architecture)

    return model

O código a seguir, contém as funções para gerar cada uma das versões da DenseNet válidas no escopo deste trabalho.

In [19]:
def DenseNet83(): 
    model =  densenet(
        architecture='densenet83', input_shape=(224, 224, 3), 
        n_classes=3, include_top=True
    )
#     print(model.summary())
    return model


def DenseNet121(): 
    model =  densenet(
        architecture='densenet121', input_shape=(224, 224, 3), 
        n_classes=3, include_top=True
    )
#     print(model.summary())
    return model


def DenseNet169(): 
    model =  densenet(
        architecture='densenet169', input_shape=(224, 224, 3), 
        n_classes=3, include_top=True
    )
#     print(model.summary())
    return model


def DenseNet201(): 
    model =  densenet(
        architecture='densenet201', input_shape=(224, 224, 3), 
        n_classes=3, include_top=True
    )
#     print(model.summary())
    return model

O código a seguir possui algumas funções para computar o número de parâmetros do modelo e o total de memória que o modelo consumirá.

In [55]:
# count number of trainable parameters in a model
def count_trainable_params(model):
    return np.sum([np.prod(v.get_shape().as_list()) for v in model.trainable_variables])


# Count number of non-trainable parameters in a model
def count_non_trainable_params(model):
    return np.sum([np.prod(v.get_shape().as_list()) for v in model.non_trainable_variables])


# Count total parameters in a model
def count_total_parametes(model):
    train_params = count_trainable_params(model)
    ntrain_params = count_non_trainable_params(model)
    
    return train_params+ntrain_params


# Count total memory usage of a model
def count_total_memory_usage(model, batch_size=4):
    return count_total_parametes(model)*batch_size/1024**2

Neste trabalho, serão criados e avaliados dois modelos baseados na DenseNet, conforme proposta no exercício 1, a DenseNet-121 e DenseNet-83. O conjunto de dados em que os modelos serão aplicados, assim como os hiper-parâmetros de compilação dos modelos serão feitos de acordo os propostos no [notebook da aula](https://github.com/vsantjr/DeepLearningMadeEasy/blob/904f86149c59aa58fa06f1d5b3a994e4ccb79b0b/PyTorch_DenseNet.ipynb).

In [22]:
# Creating DenseNet-121
densenet121 = DenseNet121()

# Parameters for compilation model
num_epochs = 10
batch_size = 4 
learning_rate = 0.001
moment = 0.9 
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=moment)
loss = tf.keras.losses.CategoricalCrossentropy()
metrics = ['accuracy']

# Compiling model
densenet121.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Printing layer arquiteture summary
densenet121.summary()

Model: "densenet121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_5 (ZeroPadding2D (None, 230, 230, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_5[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
________________________________________________________________________________________

In [49]:
# Creating DenseNet-83
densenet83 = DenseNet83()

# Parameters for compilation model
num_epochs = 10
batch_size = 4 
learning_rate = 0.001
moment = 0.9 
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=moment)
loss = tf.keras.losses.CategoricalCrossentropy()
metrics = ['accuracy']

# Compiling model
densenet83.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Printing layer arquiteture summary
densenet83.summary()

Model: "densenet83"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_6 (ZeroPadding2D (None, 230, 230, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_6[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
_________________________________________________________________________________________

O trecho de código a seguir computa o total de parâmetros e o uso de memória por cada modelo.

In [56]:
train_params_d121 = count_trainable_params(densenet121)
ntrain_params_d121 = count_non_trainable_params(densenet121)
usage_memory_d121 = count_total_memory_usage(densenet121, batch_size)

train_params_d83 = count_trainable_params(densenet83)
ntrain_params_d83 = count_non_trainable_params(densenet83)
usage_memory_d83 = count_total_memory_usage(densenet83, batch_size)

print("DenseNet-121")
print(f"\tTrainable parameters: {train_params_d121}")
print(f"\tNon-trainable parameters: {ntrain_params_d121}")
print(f"\tTotal parameters: {train_params_d121+ntrain_params_d121}")
print(f"\tMemory usage (in MB): {usage_memory_d121:.3f}")

print("DenseNet-83")
print(f"\tTrainable parameters: {train_params_d83}")
print(f"\tNon-trainable parameters: {ntrain_params_d83}")
print(f"\tTotal parameters: {train_params_d83+ntrain_params_d83}")
print(f"\tMemory usage (in MB): {usage_memory_d83:.3f}")

DenseNet-121
	Trainable parameters: 6956931
	Non-trainable parameters: 83648
	Total parameters: 7040579
	Memory usage (in MB): 26.858
DenseNet-83
	Trainable parameters: 3713351
	Non-trainable parameters: 43832
	Total parameters: 3757183
	Memory usage (in MB): 14.333
